# Identify affected buildings

In this notebook we show how to identify the affected buildings in a floodmap. We will use GeoScape buildings in ESRI ShapeFile provided by NEMA which are available [here](https://drive.google.com/drive/folders/1NH5Pu71jBxmchTKSeYyj7VF26WXLvowN?usp=share_link) (access restricted to NEMA peronell). Download the shapefiles to a local folder (e.g., [../resources/buildings](../resources/buildings) or [../flood-activations](../flood-activations)) and unzip the files.

For the purpose of this demonstration we will use the floodmap over Lismore aggregated between the 27th of March to the 15th April of 2022.

In [ ]:
# Necessary imports
import os
os.environ['USE_PYGEOS'] = '0'
import matplotlib.colors
import geopandas as gpd
from dotenv import load_dotenv
from ml4floods.data import utils
import folium

# Uncomment this to suppress deprecation warnings
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
from shapely.errors import ShapelyDeprecationWarning
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning) 

## Load environment and project details

The notebook reads the location of the GCP access key file and project name from a hidden ```.env``` file in the root directory. See [SETUP]() file for instructions on creating these.

In [ ]:
# Load environment variables (including path to credentials) from '.env' file
env_file_path = "../.env"

# Uncomment for alternative version for Windows (r"" indicates raw string)
#env_file_path = r"C:/Users/User/floodmapper/.env"

assert load_dotenv(dotenv_path=env_file_path) == True, "[ERR] Failed to load environment!"
assert "GOOGLE_APPLICATION_CREDENTIALS" in os.environ, "[ERR] Missing $GOOGLE_APPLICATION_CREDENTIAL!"
assert "GS_USER_PROJECT" in os.environ, "[ERR] Missing $GS_USER_PROJECT!"
key_file_path = os.environ["GOOGLE_APPLICATION_CREDENTIALS"]
assert os.path.exists(key_file_path), f"[ERR] Google credential key file does not exist: \n{key_file_path} "
assert "ML4FLOODS_BASE_DIR" in os.environ, "[ERR] Missing $ML4FLOODS_BASE_DIR!"
base_path = os.environ["ML4FLOODS_BASE_DIR"]
assert os.path.exists(base_path), f"[ERR] Base path does not exist: \n{base_path} "
bucket_name = os.environ["BUCKET_URI"]
assert bucket_name is not None and bucket_name != "", f"Bucket name not defined {bucket_name}"

print("[INFO] Successfully loaded FloodMapper environment.")

## Load and plot the FloodMapper-generated floodmap

Here we can load and plot a floodmap from the GCP bucket, or an example from the local disk.

In [ ]:
# Path to the floodmap to be analysed
floodmap_local_path = "../resources/floodmap_example.geojson"

# Read the floodmap and separate area-imaged polygon
#floodmap = utils.read_geojson_from_gcp(floodmap_path) # If loading a floodmap from GCP
floodmap = gpd.read_file(floodmap_local_path)
area_imaged = floodmap[floodmap["class"] == "area_imaged"].copy()
floodmap = floodmap[floodmap["class"] != "area_imaged"].copy()

# Set colors of polygon classes
COLORS = {
    'cloud': "gray",
    'flood_trace': "turquoise",
    'water': "blue"
}
categories = floodmap["class"].unique()
print("[INFO] Categories in file: ", categories)
cmap = matplotlib.colors.ListedColormap([COLORS[b] for b in categories])

# Plot the floodmap bounded by the AoI
m = floodmap.explore(column="class", cmap=cmap,categories=categories)
m = area_imaged.explore(m=m,color="red",style_kwds={"fill":False})

In [ ]:
m

## Load the Geoscape building footprints 

The GeoScape building footprints are saved in the 'EPSG:7844' coordinate reference system (CRS), so we to convert the floodmap to this CRS before comparing. For convenience, we also subset the floodmap `GeoDataFrame` object to keep only the `water` and `flood_trace` polygons.

In [ ]:
# Convert floodmap and area imaged polygons to matching CRS
floodmap_crs7844 = floodmap.to_crs("EPSG:7844")
area_imaged_crs7844 = area_imaged.to_crs("EPSG:7844")

# Filter the floodmap for 'water' and 'flood_trace' polygons (drop clouds, if present)
floodmap_crs7844_innundated = floodmap_crs7844[floodmap_crs7844["class"].isin(["water","flood_trace"])]

In the next cell we read the GeoScape buildings over the area of interest. For this we use the `read_file` function of `geopandas` and specify a bounding box (`bbox` argument) to read. This makes the reading of the buildings much faster, as we only load the buildings in this bounding box instead of all buildings in the file.

In [ ]:
%%time

# Set the path to the buildings file
buildings_path = "../resources/buildings/Buildings/Buildings JUNE 2022/Standard/nsw_buildings.shp"

# Load the building polygons for the mapped area only
bounding_box = tuple(area_imaged_crs7844.iloc[0].geometry.bounds)
buildings_in_flood_area = gpd.read_file(buildings_path, bbox=bounding_box)
buildings_in_flood_area.head()

## Compute and display the affected buildings

In the next cell we intersect the buildings and the floodmap. We add a column in the buildings dataframe that indicates if the building intersect the floodmap (i.e., if it has been affected by the flooding). The operation may take a few seconds to perform on larger maps.

In [ ]:
%%time

# Add column whether the building has been affected by the flood
buildings_in_flood_area["affected"] = buildings_in_flood_area.geometry.apply(
    lambda x: "yes" if floodmap_crs7844_innundated.intersects(x).any() else "no")

In the next cell we overlay the buildings and flood-extent map, colour-coding the affected buildings in red and the non-affected buildings in gray.

In [ ]:
categories_buildings = ["yes","no"]
cmap_buildings = matplotlib.colors.ListedColormap(["#DD0000","#888888"])

# First plot the floodmap
m = floodmap.explore(column="class", cmap=cmap, categories=categories, 
                     style_kwds={"fillOpacity":0.25}, legend=False, name="Floodmap")

# Then plot the affected buildings
m = buildings_in_flood_area[["geometry","affected","CAPT_DATE","AREA","GRD_ELEV"]].explore(
    column="affected", cmap=cmap_buildings, categories=categories_buildings,
    style_kwds={"fillOpacity":0.25}, legend=False, name="Buildings", m=m)

# Then overlay the floodmap and AoI bounds
m = area_imaged.explore(m=m, color="red", style_kwds={"fill": False}, name="AoI")

# Add the layer control and show
folium.LayerControl(collapsed=False).add_to(m)
m

## Statistics of affected buildings

In [ ]:
buildings_in_flood_area_affected = buildings_in_flood_area[buildings_in_flood_area.affected == "yes"]

print(f"There are {buildings_in_flood_area_affected.shape[0]} affected buildings")
print(f"Area of the affected buildings: {buildings_in_flood_area_affected.AREA.sum()} m²")